# Convert custom data using PyNWB

Previously you learned how to convert electrophysiology data from a common data type (Neuropixels) to the NWB format, using the [GUIDE](https://nwb-guide.readthedocs.io/en/latest).

In this tutorial we'll learn how to use [PyNWB](https://pynwb.readthedocs.io/en/stable/index.html) to add custom trials data to an existing NWB file. 

Let's start by installing the necessary packages.

In [1]:
%%bash

pip install -U pynwb dandi pyyaml nwbinspector pandas

# Download NWB file from DANDI archive (staging)

To download the dandiset containing the NWB files, we can use the [dandi cli](https://www.dandiarchive.org/handbook/12_download/#using-the-python-cli-client). The command below will create a folder in your current working directory containing all NWB files from the chosen dandiset. 

Remember to **change the dandiset id number**, choose the dandiset you created for the GUIDE tutorial.

In [2]:
# Set your dandiset number here
dandiset_id = "213840"

# Run the bash command
bash_command = f'dandi download https://gui-staging.dandiarchive.org/dandiset/{dandiset_id}/draft'

!{bash_command}

PATH                                               SIZE     DONE           DONE% CHECKSUM STATUS          MESSAGE   
213840/dandiset.yaml                                                                      done            updated   
213840/sub-id0123/sub-id0123_ses-id987_ecephys.nwb 18.3 MB  18.3 MB         100%    ok    done                      
Summary:                                           18.3 MB  18.3 MB                       2 done          1 updated 
                                                            100.00%                                                 
2024-05-23 15:52:40,495 [    INFO] Logs saved in /home/luiz/.local/state/dandi-cli/log/20240523135227Z-338388.log


## Open Downloaded file and explore it using PyNWB

Let's open the downloaded file with PyNWB and verify its contents. Remember to **change the path** to the file you just downloaded.

In [3]:
import pynwb


# Change this to the path to the NWB file you just downloaded
file_path = "213840/sub-id0123/sub-id0123_ses-id987_ecephys.nwb"

# Open the file with PyNWB and verify its contents
read_io = pynwb.NWBHDF5IO(file_path, mode="r")
nwbfile = read_io.read()
nwbfile

,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP2,20.0,square,0.0,2.0,0.076923
3,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP3,20.0,square,32.0,3.0,0.076923
,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921265251360\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921265251024\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP2,20.0,square,0.0,2.0,0.076923


In [4]:
# You can access all data and information inside the nwbfile object
print("Electrical series recording information:")
print(nwbfile.acquisition['ElectricalSeriesAP'])

print("Subject information:")
print(nwbfile.subject)

Electrical series recording information:
ElectricalSeriesAP pynwb.ecephys.ElectricalSeries at 0x139921265249296
Fields:
  comments: no comments
  conversion: 2.34375e-06
  data: <HDF5 dataset "data": shape (90000, 384), type "<i2">
  description: Acquisition traces for the ElectricalSeriesAP.
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  offset: 0.0
  rate: 30000.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: volts

Subject information:
subject pynwb.file.Subject at 0x139921265251840
Fields:
  age__reference: birth
  date_of_birth: 2024-01-01 11:05:00
  sex: M
  species: Mus musculus
  subject_id: id0123



In [5]:
# Tabular information such as electrodes or sorted units can be explored as Pandas dataframes
nwbfile.electrodes.to_dataframe()

,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP2,20.0,square,0.0,2.0,0.076923
3,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP3,20.0,square,32.0,3.0,0.076923
4,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP4,40.0,square,16.0,4.0,0.153846
...,...,...,...,...,...,...,...,...,...
379,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP379,3780.0,square,32.0,379.0,0.692308
380,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP380,3800.0,square,16.0,380.0,0.769231
381,unknown,s0 pynwb.ecephys.ElectrodeGroup at 0x139921265...,s0,AP381,3800.0,square,48.0,381.0,0.769231


In [6]:
# Let's check the Units table
nwbfile.units.to_dataframe()

,spike_times,unit_name,quality,channel_group,original_cluster_id
id,,,,,
0,"[0.0426, 0.1105, 0.11576666666666667, 0.119933...",0,unsorted,0.0,0.0
1,"[0.18046666666666666, 0.23866666666666667, 0.4...",1,unsorted,0.0,1.0
2,"[0.07986666666666667, 0.18796666666666667, 0.1...",2,unsorted,0.0,2.0
3,"[0.0197, 0.09153333333333333, 0.31053333333333...",3,unsorted,0.0,3.0
4,"[0.007633333333333333, 0.05383333333333333, 0....",4,unsorted,0.0,4.0
5,"[0.0028333333333333335, 0.007266666666666667, ...",5,unsorted,0.0,5.0
6,"[0.025133333333333334, 0.20776666666666666, 0....",6,unsorted,0.0,6.0
7,"[0.030633333333333332, 0.037233333333333334, 0...",7,unsorted,0.0,7.0
8,"[0.013166666666666667, 0.3263, 0.4342, 0.484, ...",8,unsorted,0.0,8.0


When opening a file with PyNWB without a context manager (as we've just done in the code above) it is always important to **close the file**.

In [7]:
# Remember to close the file if you are exploring it outside of the context manager mode
read_io.close()

# Add data to the original NWB file

The NWB file you created for the GUIDE tutorial contains raw electrophysiology traces and processed spiking data. With PyNWB you can include more experimental data to the file.

Next, you will add trials information to this file. Read more about adding and removing data from NWB files [here](https://pynwb.readthedocs.io/en/stable/tutorials/general/add_remove_containers.html). In this session we will:
- Open the NWB file in append mode.
- Add trials data. Read more about time intervals in NWB [here](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#time-intervals).
- Write the modified NWB file.
- Open the file and explore the new content in it
- Close the file

In [8]:
# Open the file in `r+` mode, that allows us to add new data to it
with pynwb.NWBHDF5IO(file_path, mode="a") as io:
    nwbfile = io.read()

    # Create trials data and add them to the NWBFile object
    nwbfile.add_trial_column(
        name="correct",
        description="whether the trial was correct",
    )
    nwbfile.add_trial(start_time=0.0, stop_time=5.0, correct=True)
    nwbfile.add_trial(start_time=5.0, stop_time=10.0, correct=False)
    nwbfile.add_trial(start_time=10.0, stop_time=15.0, correct=True)

    # write the modifications to disk
    io.write(nwbfile)

In [9]:
# Read the NWB file and confirm the modifications
read_io = pynwb.NWBHDF5IO(file_path, mode="r")
nwbfile = read_io.read()
nwbfile

,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP2,20.0,square,0.0,2.0,0.076923
3,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP3,20.0,square,32.0,3.0,0.076923
,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x139921271861504\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x139921271861312\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP2,20.0,square,0.0,2.0,0.076923


In [10]:
# Let's check the newly created Trials table
nwbfile.trials.to_dataframe()

,start_time,stop_time,correct
id,,,
0,0.0,5.0,True
1,5.0,10.0,False
2,10.0,15.0,True


In [11]:
# Remember to close the file if you are exploring it outside of the context manager mode
read_io.close()

# Create a new NWB file based on the original NWB file
Many times it can be useful to modify the original NWB file, e.g. to include new processed data or to exclude raw data.

Next, you will create a new NWB file based on the original file, add new processed data and remove the raw traces from it. Read more about adding and removing data from NWB files [here](https://pynwb.readthedocs.io/en/stable/tutorials/general/add_remove_containers.html). In this session we will:
- Open the NWB file in read mode.
- Remove the raw electrophysiology traces.
- Add mock processed behavioral data. Read more about behavioral data in NWB [here](https://pynwb.readthedocs.io/en/stable/tutorials/domain/plot_behavior.html).
- Write the modified NWB object as a new NWB file.
- Open the new file and explore the content in it
- Close the file

In [12]:
from utils import get_file_size_in_mb
import numpy as np


# Open the file in `r` mode, generate new unique id
with pynwb.NWBHDF5IO(file_path, mode="r") as read_io:
    nwbfile = read_io.read()
    nwbfile.generate_new_id()

    # Remove the raw traces from the NWBFile object
    nwbfile.acquisition.pop("ElectricalSeriesAP")

    # Create a Behavior Processing Module
    behavior_module = nwbfile.create_processing_module(
        name="behavior",
        description="Processed behavioral data"
    )
    
    # Add mock processed behavioral data
    speed_data = np.random.random(15)
    speed_time_series = pynwb.TimeSeries(
        name="speed",
        data=speed_data,
        rate=1.,
        description="The speed of the subject measured over time.",
        unit="m/s",
    )
    behavioral_time_series = pynwb.behavior.BehavioralTimeSeries(
        time_series=speed_time_series,
        name="BehavioralTimeSeries",
    )
    
    behavior_module.add(behavioral_time_series)

    # Use the export method to write the modified NWBFile object to a new file path. 
    # The original file is not modified
    # file_path_modified = file_path.split(".nwb")[0] + "_modified.nwb"
    file_path_modified = "modified_file.nwb"
    with pynwb.NWBHDF5IO(file_path_modified, mode="w") as export_io:
        export_io.export(src_io=read_io, nwbfile=nwbfile)

    print(f"Original file size: {get_file_size_in_mb(file_path)} MB")
    print(f"Modified file size: {get_file_size_in_mb(file_path_modified)} MB")

Original file size: 17.46 MB
Modified file size: 0.3 MB


In [13]:
# Read the modified NWB file and confirm the modifications
read_io = pynwb.NWBHDF5IO(file_path_modified, mode="r")
nwbfile = read_io.read()
nwbfile

root pynwb.file.NWBFile at 0x139921246075728
Fields:
  devices: {
    Neuropixel-Imec <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    s0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Created for tutorial
  experimenter: ['Tauffer, Luiz']
  file_create_date: [datetime.datetime(2024, 5, 6, 11, 8, 14, 836458, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))]
  identifier: a9cefea4-ef02-45de-a2db-936215efcf2b
  institution: My Uni
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  keywords: <StrDataset for HDF5 dataset "keywords": shape (1,), type "|O">
  lab: My lab
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  session_description: Auto-generated by neuroconv
  session_id: id987
  session_start_time: 2024-05-06 10:56:41
  subject: subject pynwb.file.Subject at 0x139921246073232
Fields:
  age__reference: birth
  date_of_birth: 2024-01-01 11:05:00
  sex: M
  species: Mus musculus
  subject_id: id0123

  timestamps_reference_time: 2024-05-06 10:56:41
  trials: trials <class 'pynwb.epoch.TimeIntervals'>
  units: units <class 'pynwb.misc.Units'>

In [14]:
# Remember to close the file if you are exploring it outside of the context manager mode
read_io.close()

# Upload the modified NWB files to DANDI archive (staging)

Now we can upload the modified NWB files to DANDI archive. Let's upload them to the same dandiset you've been using, for simplicity.

In this session, you will use the [dandi cli](https://www.dandiarchive.org/handbook/12_download/#using-the-python-cli-client) to:
- Organize the files within the dandiset, which will automatically name the files copies using the DANDI convention.
- Validate the files
- Upload the files to DANDI archive. Don't forget to **set the right value for your DANDI_API_KEY** in the code below.

In [15]:
bash_command = f'dandi organize -d {dandiset_id} -f move --required-field session_id .'

!{bash_command}

2024-05-23 15:53:01,180 [    INFO] Loading metadata from 2 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    3.9s finished
2024-05-23 15:53:05,106 [    INFO] Organized 1 out of 2 paths (1 same existing skipped). Visit 213840/
2024-05-23 15:53:05,107 [    INFO] Logs saved in /home/luiz/.local/state/dandi-cli/log/20240523135254Z-339011.log


In [16]:
bash_command = f'dandi validate {dandiset_id}'

!{bash_command}

No errors found.
2024-05-23 15:53:20,987 [    INFO] Logs saved in /home/luiz/.local/state/dandi-cli/log/20240523135306Z-339345.log


In [17]:
import os


# Save your API key for Dandi staging as an environment variable
os.environ["DANDI_API_KEY"] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Run the bash command
bash_command = f'dandi upload -i dandi-staging {dandiset_id}'

!{bash_command}

2024-05-23 15:53:29,451 [    INFO] Found 3 files to consider
PATH                                                 SIZE     ERRORS  PROGRESS STATUS                MESSAGE                       
dandiset.yaml                                        1.2 kB                    skipped               should be edited online       
sub-id0123/sub-id0123_ses-id987_behavior+ecephys.nwb 312.0 kB   0         100% done                                                
sub-id0123/sub-id0123_ses-id987_ecephys.nwb          18.3 MB    0         100% done                  exists (older) - reuploading  
Summary:                                             18.6 MB          1.4 MB/s 1 skipped             1 should be edited online     
                                                                               2 done                1 exists (older) - reuploading
2024-05-23 15:53:42,576 [    INFO] Logs saved in /home/luiz/.local/state/dandi-cli/log/20240523135321Z-339569.log


# Explore the modified data files in DANDI archive and Neurosift

Once the modified data has been uploaded to DANDI archive, it becomes immediately available to be read and visualized online!

For that we will use [Neurosift](https://neurosift.app), a web app that allows you to explore and visualize all the data stored in the DANDI archive. Follow these steps:
- Run the cell below to get the url for your dandiset.
- Navigate to `Files` under `Dandiset Actions`
- Over the modified file, click on `Open with` then `Neurosift`

Alternatively, you can open [Neurosift](https://neurosift.app) and navigate through all available dandisets directly from there!

In [18]:
import yaml

# Get the dandiset url from the YAML file
yaml_file_path = f"{dandiset_id}/dandiset.yaml"
with open(yaml_file_path, 'r') as file:
    yaml_dict = yaml.safe_load(file)
    print(yaml_dict["url"])

https://gui-staging.dandiarchive.org/dandiset/213840/draft
